# Introdution

In this notebook we intend to develope a multi Input tensorflow model for the given data set

Note: The given data set is unbalanced ie number of positives small as compared to negatives.
Hence we will develope seperate models, other than neural networks, which deals with these imbalance
by synthetically enhancing data using techinques like smote etc

In [ ]:
# Making essential imports
import numpy as np
import pandas as pd 
import seaborn as sb
import tensorflow as tf
import os
import matplotlib.pyplot as plt

# printing directory contents
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print( 'Data path => '+ os.path.join(dirname, filename))


# loading data
dataFrame = pd.read_csv('/kaggle/input/pulsar-dataset-htru2/HTRU_2.csv')
dataFrame.info()

For our convinence lets map the columns to their respective name in the data set

In [ ]:
maper = {
    '140.5625' : 'Mean of integrated profile',
    '55.68378214' : 'Standard Deviation of integrated profile',
    '-0.234571412' : 'Excess kurtosis of the integrated profile',
    '-0.699648398' : 'Skewness of the integrated profile',
    '3.199832776'  : 'Mean of the DM-SNR curve',
    '19.11042633'  : 'Standard deviation of the DM-SNR curve',
    '7.975531794'  : 'Excess kurtosis of the DM-SNR curve',
    '74.24222492'  :'Skewness of the DM-SNR curve',
    '0' : 'Target'
}
dataFrame = dataFrame.rename(columns = maper )
dataFrame[0:10]

# Visualizing Data Set
For Visualizing your data perform the following sets:

1.) select the features & targets from your dataset

2.) Do standard scalling (this step is important)

3.) Do pca Clustering and convert the resultant data to dataFrame

4.) use seaborn for visualizing this data


In [ ]:
from sklearn.preprocessing import StandardScaler
from  sklearn.decomposition import PCA

features = list (dataFrame.columns[0:-1])
featureVector = dataFrame[features]
targets = dataFrame['Target']

# Scalling data
stdSc = StandardScaler()
featureVector = stdSc.fit_transform(featureVector)

# pca clustering
pca = PCA(n_components = 2)
pca.fit(featureVector)
featureVectorT = pca.transform(featureVector)

#Converting to dataset
dimReducedDataFrame = pd.DataFrame(featureVectorT)
dimReducedDataFrame = dimReducedDataFrame.rename(columns = {0: 'V1', 1 : 'V2'})
dimReducedDataFrame['Target'] = targets

# Plotting
plt.figure(figsize = (10, 7))
sb.scatterplot(data = dimReducedDataFrame, x = 'V1', y = 'V2', hue = 'Target')
plt.grid(True)
plt.show()


Well this is a good sign as you can see there exists
a  decission boundary, excluding some outliers, to what I believe
a simple tensorflow model should be able to perform well on this dataset.
Let's see if this is true.

Also just for fun a model predicting every signal as non - pulsar will have an accuracy of 90%
so I believe we shall try to achieve an accuracy of more than 90 %

In [ ]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(featureVector, targets, test_size = 0.1, random_state = 42)
m,n = xTrain.shape
# defining model
simpleAnn = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape = (n, )),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dropout(0.07),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

simpleAnn.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy',
                                                                              tf.keras.metrics.AUC()])

In [ ]:
retVal = simpleAnn.fit(xTrain, yTrain, validation_data = (xTest, yTest), batch_size = 100, epochs = 50, verbose = 1)

In [ ]:
plt.plot(retVal.history['loss'], label = 'training loss')
plt.plot(retVal.history['accuracy'], label = 'training accuracy')
plt.plot(retVal.history['auc'], label = 'auc_train')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
plt.plot(retVal.history['val_loss'], label = 'validation loss')
plt.plot(retVal.history['val_accuracy'], label = 'validation accuracy')
plt.plot(retVal.history['val_auc'], label = 'auc_validation')
plt.grid(True)
plt.legend()
plt.show()

98 % validation accuracy, let's see what will happen if use pca clustered data for our testing

In [ ]:
# I am copy pasting the code, this is a bad practice avoid it at all cost
# use functions, unless you want to save time
from sklearn.model_selection import train_test_split
xTrain1, xTest1, yTrain1, yTest1 = train_test_split(featureVectorT, targets, test_size = 0.1, random_state = 42)
m,n = xTrain.shape
# defining model
simpleAnnPCA = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape = (n, )),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dropout(0.07),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

simpleAnnPCA.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy',
                                                                              tf.keras.metrics.AUC()])

In [ ]:
retVal = simpleAnnPCA.fit(xTrain, yTrain, validation_data = (xTest, yTest), batch_size = 100, epochs = 50, verbose = 1)

again results in acceptable ranges let us now do what the title of notebook says
design a multi-Input TensorFlow model.

In [ ]:
inp1 = tf.keras.layers.Input(shape = ((featureVector.shape[1], )), name = 'inp1')
inp2 = tf.keras.layers.Input(shape = ((featureVectorT.shape[1], )), name = 'inp2')
D1 = tf.keras.layers.Dense(32, activation = 'relu')(inp1)
D2 = tf.keras.layers.Dense(32, activation = 'relu')(inp2)
ConcLayer = tf.keras.layers.concatenate([D1, D2])
D3 = tf.keras.layers.Dense(64, activation = 'relu')(ConcLayer)
D4 = tf.keras.layers.Dense(64, activation = 'relu')(D3)
D5 = tf.keras.layers.Dropout(0.07)(D4)
output = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'opt')(D3)

Model = tf.keras.Model(inputs = [inp1, inp2], outputs = [output])
Model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy', tf.keras.metrics.AUC()])
Model.summary()


In [ ]:
import keras
keras.utils.plot_model(Model, "MultiHeadModel.png", show_shapes=True)

In [ ]:
# let train
retVal = Model.fit({'inp1' : featureVector, 'inp2' : featureVectorT},  np.array(targets, dtype = 'int'), epochs = 50)

As You can see the multi-Head / ensembled network  works better than the other 2.
Hence Enmebling is the key to victory.


Thanks, if you Reached here, I hope this notebook was helpful.